# Track and Count Objects Using ByteTrack ✅

In [1]:
! nvidia-smi

Mon Mar 11 08:01:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


# Importing Video to the Environment

In [3]:
%cd {HOME}
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-" -O vehicle-counting.mp4 && rm -rf /tmp/cookies.txt

/content
--2024-03-11 08:02:54--  https://docs.google.com/uc?export=download&confirm=&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-
Resolving docs.google.com (docs.google.com)... 172.217.218.138, 172.217.218.100, 172.217.218.139, ...
Connecting to docs.google.com (docs.google.com)|172.217.218.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-&export=download [following]
--2024-03-11 08:02:55--  https://drive.usercontent.google.com/download?id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 108.177.127.132, 2a00:1450:4013:c07::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|108.177.127.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35345757 (34M) [video/mp4]
Saving to: ‘vehicle-counting.mp4’

vehicle-counting.mp 100%[===================>]  33.71M

# Load and Install YOLOv8

In [4]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.26 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.4/78.2 GB disk)


In [5]:
MODEL = "yolov8x.pt"
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

100%|██████████| 131M/131M [00:00<00:00, 293MB/s]


YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


In [6]:
!yolo task=detect mode=predict model=yolov8x.pt conf=0.25 source={"/content/vehicle-counting.mp4"}

Ultralytics YOLOv8.1.26 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs

video 1/1 (frame 1/538) /content/vehicle-counting.mp4: 384x640 3 cars, 1 truck, 122.7ms
video 1/1 (frame 2/538) /content/vehicle-counting.mp4: 384x640 3 cars, 1 truck, 51.9ms
video 1/1 (frame 3/538) /content/vehicle-counting.mp4: 384x640 4 cars, 2 trucks, 51.9ms
video 1/1 (frame 4/538) /content/vehicle-counting.mp4: 384x640 5 cars, 1 truck, 62.0ms
video 1/1 (frame 5/538) /content/vehicle-counting.mp4: 384x640 4 cars, 1 truck, 62.0ms
video 1/1 (frame 6/538) /content/vehicle-counting.mp4: 384x640 4 cars, 1 truck, 62.0ms
video 1/1 (frame 7/538) /content/vehicle-counting.mp4: 384x640 4 cars, 1 truck, 62.0ms
video 1/1 (frame 8/538) /content/vehicle-counting.mp4: 384x640 5 cars, 1 truck, 62.0ms
video 1/1 (frame 9/538) /content/vehicle-counting.mp4: 384x640 4 cars, 1 truck, 62.0ms
video 1/1 (frame 10/538) /content/vehicle-c

In [7]:
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names
# class_ids of interest - car, motorcycle, bus and truck
CLASS_ID = [0,1,2, 3, 5, 7,9]

# Installing Roboflow Supervision

In [8]:
!pip install supervision


from IPython import display
display.clear_output()


import supervision
print("supervision.__version__:", supervision.__version__)


supervision.__version__: 0.18.0


# ✅ Track and Count Algorithm

In [15]:
import supervision as sv
import numpy as np
from ultralytics import YOLO
import cv2

# Define the model and tracker
MODEL = "yolov8x.pt"
model = YOLO(MODEL)
tracker = sv.ByteTrack()

# Define the bounding box annotator and label annotator with custom thickness and font size
bounding_box_annotator = sv.BoundingBoxAnnotator(thickness=6)  # Increase thickness to 3 pixels
label_annotator = sv.LabelAnnotator(text_thickness= 3, text_scale=3)  # Increase font size by a factor of 1.2

#Defining line co-ordinates for counting objects crossing the line
LINE_START = sv.Point(50, 1500)
LINE_END = sv.Point(3840-50, 1500)

#Initialize Line Annotator with custom thickness and font size
line_zone = sv.LineZone(start=LINE_START , end=LINE_END)
line_annotator = sv.LineZoneAnnotator(thickness=3, text_thickness=3, text_scale=3)

# Define the callback function
def callback(frame: np.ndarray, index: int) -> np.ndarray:

    # Perform object detection
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)

    # Update tracker with detections
    detections = tracker.update_with_detections(detections)


    # Extract labels for annotations
    labels = []
    for tracker_id, class_id in zip(detections.tracker_id, detections.class_id):
        class_name = CLASS_NAMES_DICT[class_id]
        labels.append(f"#{tracker_id} - {class_name}")


    # Annotate bounding boxes on the frame
    annotated_frame = bounding_box_annotator.annotate(scene=frame.copy(), detections=detections)

    # Annotate labels on the frame
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)

    #Draw the Line
    line_annotator.annotate(annotated_frame, line_zone)

    #Trigger the Line Annotation for Updating Counter
    line_zone.trigger(detections=detections)

    return annotated_frame

# Process the video
sv.process_video(
    source_path="/content/vehicle-counting.mp4",
    target_path="/content/final_result.mp4",
    callback=callback
)


0: 384x640 3 cars, 1 truck, 51.3ms
Speed: 4.0ms preprocess, 51.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 49.4ms
Speed: 3.2ms preprocess, 49.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 2 trucks, 44.0ms
Speed: 3.1ms preprocess, 44.0ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 truck, 44.2ms
Speed: 3.1ms preprocess, 44.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 43.8ms
Speed: 3.1ms preprocess, 43.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 43.8ms
Speed: 3.1ms preprocess, 43.8ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 40.6ms
Speed: 3.1ms preprocess, 40.6ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 truck, 40.5ms
Speed: 3.1ms preprocess, 40.5ms 

In [16]:
video_info = sv.VideoInfo.from_video_path(video_path='/content/final_result.mp4')
video_info

VideoInfo(width=3840, height=2160, fps=25, total_frames=538)

# Wohoo! It's Done!! 🏆😀